# Google Cloud の生成 AI サービス（言語モデル）を LangChain から使用するサンプル

**[注意]**

このノートブックは、Python 3.8 の環境に Vertex AI の SDK と LangChain のライブラリーを事前インストールした Kernel が必要です。具体的には、次の手順で用意します。

1. Workbench のターミナルから次のコマンドを実行します。

```
conda create -n python38 python=3.8 -y
conda activate python38
conda install ipykernel -y
ipython kernel install --user --name=python38
pip install google-cloud-aiplatform==1.28.1 --upgrade --user
pip install langchain==0.0.250 --user
conda deactivate
```

2. ブラウザーをリロードして、ローンチャーを開くと ［Python38(Local)］のボタンが追加されているので、このボタンから新規のノートブックを開きます。


## LangChain を用いて、基本的なパインプラインを実行する例

新製品の説明文から商品名のアイデアを3つ出してもらうためのプロンプトを定義します。

In [1]:
from langchain import PromptTemplate

template = """\
You are a naming consultant for new products.
Give me three examples of good names for a product.
Output only names in a comma-separated list, nothing else.

Here's an example.
product: cute pens
output: Scribble, Ink-it, Write-On

Here's the real request to you.
product: {description}
output:
"""

prompt = PromptTemplate(template=template, input_variables=['description'])

In [2]:
print(prompt.format(description='Colorful cute smartphone covers for teenagers'))

You are a naming consultant for new products.
Give me three examples of good names for a product.
Output only names in a comma-separated list, nothing else.

Here's an example.
product: cute pens
output: Scribble, Ink-it, Write-On

Here's the real request to you.
product: Colorful cute smartphone covers for teenagers
output:



Vertex AI の LLM API を呼び出すクライアントオブジェクトを取得します。

In [3]:
from langchain.llms import vertexai
llm = vertexai.VertexAI(model_name='text-bison', temperature=0.4)

パイプラインを定義して実行します。

In [4]:
from langchain import LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [5]:
description = 'Colorful cute smartphone covers for teenagers'
print(llm_chain.run(description))

PhoneCase, PhoneCover, PhoneShell


LLM からの応答文をリスト形式に変換します。

In [6]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

In [7]:
output_parser.parse(llm_chain.run(description))

['1. Case-O-Color', '2. Phone-Cover-Fun', '3. Cover-Me-Up']

## pydantic を利用して、LLM からの応答をオブジェクトに変換する例

LLM からの応答を格納するオブジェクトを定義して、対応するパーサーを生成します。

In [8]:
from pydantic import BaseModel, Field

class ProductNames(BaseModel):
    setup: str = Field(description='product description')
    product_name1: str = Field(description='product name 1')
    product_name2: str = Field(description='product name 2')
    product_name3: str = Field(description='product name 3')

In [9]:
from langchain.output_parsers import PydanticOutputParser
parser =  PydanticOutputParser(pydantic_object=ProductNames)

In [10]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "product description", "type": "string"}, "product_name1": {"title": "Product Name1", "description": "product name 1", "type": "string"}, "product_name2": {"title": "Product Name2", "description": "product name 2", "type": "string"}, "product_name3": {"title": "Product Name3", "description": "product name 3", "type": "string"}}, "required": ["setup", "product_name1", "product_name2", "product_name3"]}
```


パーサーが生成するインストラクションを含んだプロンプトを定義した後、パイプラインを再構成して実行します。

In [11]:
template="""\
Answer the user query.
{format_instructions}
You are a naming consultant for new products.
Give me examples of good names for a product described as {description}'
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['description'],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [12]:
output = parser.parse(llm_chain.run(description))
output

ProductNames(setup="Colorful cute smartphone covers for teenagers'", product_name1='Cute Phone Cases', product_name2='Fun Phone Cases', product_name3='Trendy Phone Cases')

In [13]:
output.product_name1, output.product_name2, output.product_name3

('Cute Phone Cases', 'Fun Phone Cases', 'Trendy Phone Cases')